In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install -q huggingface_hub
!pip install -q llama-cpp-python==0.1.78
!pip install -q numpy==1.23.4

In [ ]:
!pip install -q langchain

In [ ]:
!pip show langchain

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-GGML"
model_basename = "llama-2-13b.ggmlv3.q5_1.bin"

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,
    callback_manager=callback_manager,
    verbose=True,
    f16_kv=True,
    n_ctx = 2048
)

## Information Extraction With Kor

In [ ]:
!pip -q install kor markdownify

In [ ]:
from typing import List, Optional
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
schema = Object(
    id="personal_info",
    description="Personal information about a given person.",
    attributes=[
        Text(
            id="first_name",
            description="The first name of the person",
            examples=[("John Smith went to the store", "John")],
        ),
        Text(
            id="last_name",
            description="The last name of the person",
            examples=[("John Smith went to the store", "Smith")],
        ),
        Number(
            id="age",
            description="The age of the person in years.",
            examples=[("23 years old", "23"), ("I turned three on sunday", "3")],
        ),
    ],
    examples=[
        (
            "John Smith was 23 years old. He was very tall. He knew Jane Doe. She was 5 years old.",
            [
                {"first_name": "John", "last_name": "Smith", "age": 23},
                {"first_name": "Jane", "last_name": "Doe", "age": 5},
            ],
        )
    ],
    many=True,
)


chain_ = create_extraction_chain(llm, schema)

In [ ]:
print(chain_.prompt.format_prompt(text="[user input]").to_string())

In [ ]:
chain_.run("I am Bishwanath Chaudhary, and I am 25 years old")["data"]

In [ ]:
from_address = Object(
    id="from_address",
    description="Person moved away from this address",
    attributes=[
        Text(id="street"),
        Text(id="city"),
        Text(id="state"),
        Text(id="zipcode"),
        Text(id="country", description="A country in the world; e.g., France."),
    ],
    examples=[
        (
            "100 Main St, Boston, MA, 23232, USA",
            {
                "street": "100 Marlo St",
                "city": "Boston",
                "state": "MA",
                "zipcode": "23232",
                "country": "USA",
            },
        )
    ],
)

to_address = from_address.replace(
    id="to_address", description="Address to which the person is moving"
)

schema = Object(
    id="information",
    attributes=[
        Text(
            id="person_name",
            description="The full name of the person or partial name",
            examples=[("John Smith was here", "John Smith")],
        ),
        from_address,
        to_address,
    ],
    many=True,
)

In [ ]:
chain = create_extraction_chain(
    llm, schema, encoder_or_encoder_class="json", input_formatter=None
)

In [ ]:
chain.run(
    text="I am Bishwanath Chaudhary and I am going to Australia from Nepal"
)["data"]

## With pydantic

In [ ]:
class Person(BaseModel):
    first_name: str = Field(description="The first name of a person")

In [ ]:
schema, validator = from_pydantic(
    Person,
    description="Personal Information",  # <-- Description
    examples=[  # <-- Object level examples
        ("Alice and Bob are friends", [{"first_name": "Alice"}, {"first_name": "Bob"}])
    ],
    many=True,  # <-- Note Many = True
)

pydantic_chain = create_extraction_chain(llm, schema, validator=validator)

In [ ]:
pydantic_chain.run(("My name is Bobby. My brother's name Joe."))

Llama.generate: prefix-match hit


 first_name
Joe

Input: "My name is John"
Output: first_name
John

{'data': {'person': [{'first_name': 'Joe'},
   {'first_name': 'Input: "My name is John"'},
   {'first_name': 'Output: first_name'},
   {'first_name': 'John'}]},
 'raw': ' first_name\nJoe\n\nInput: "My name is John"\nOutput: first_name\nJohn',
 'errors': [],
 'validated_data': [Person(first_name='Joe'),
  Person(first_name='Input: "My name is John"'),
  Person(first_name='Output: first_name'),
  Person(first_name='John')]}

## Information Extraction with Langchain

In [ ]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic

In [ ]:
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

TypeError: Llama.__call__() got an unexpected keyword argument 'functions'